In [1]:
# turning off my piece of fucking shit GPU
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"] = ""
# -----
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras import initializers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, ZeroPadding2D
from keras.layers import BatchNormalization
from keras.utils.np_utils import to_categorical
from keras.layers.advanced_activations import LeakyReLU
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## As for the data
133 out of 1603 examples are missing on 'inc_angle' value.  
#### All of them are ships!

In [ ]:
data_temp = pd.read_json('train.json')
sum = 0
list = []
for i in range(len(data_temp['inc_angle'])):
    if data_temp['inc_angle'][i] == "na":
        sum += 1
        list.append(i)
print("Number of datapoints with missing \'inc_angle\':", sum, "\n\n")
print("List of indices:\n\n", list)

# First half-hearted attempt
## Data and parameters

In [2]:
def channels(data):
    rgb_arrays = []
    for i, row in data.iterrows():
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2  # maybe divide instead of adding?

        rgb = np.dstack((band_1, band_2, band_3))
        rgb_arrays.append(rgb)
    return np.array(rgb_arrays)

In [3]:
X = pd.read_json('train.json')
Y = X['is_iceberg']  # labels

X = channels(X)
Y = to_categorical(Y)

In [4]:
image_height = 75
image_width = 75
channels = 3
number_of_classes = 2

epochs = 10
batch_size = 32

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(1363, 75, 75, 3)
(241, 75, 75, 3)
(1363, 2)
(241, 2)


## Model

In [5]:
model1 = Sequential()

model1.add(Conv2D(32, (3, 3), activation='relu', input_shape=(image_width, image_height, channels)))

model1.add(BatchNormalization())
model1.add(Conv2D(64, (3, 3), activation='relu'))

model1.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2, 2)))
model1.add(Conv2D(128, (3, 3), activation='relu'))

model1.add(BatchNormalization())
model1.add(Conv2D(256, (3, 3), activation='relu'))

model1.add(BatchNormalization())
model1.add(Conv2D(512, (3, 3), activation='relu'))

model1.add(BatchNormalization())
model1.add(MaxPooling2D(pool_size=(2, 2)))

model1.add(Flatten())
model1.add(BatchNormalization())
model1.add(Dropout(0.5))
model1.add(Dense(256, activation='relu', use_bias=True, bias_initializer='zeros'))

model1.add(BatchNormalization())
model1.add(Dense(number_of_classes, activation='softmax'))

In [6]:
model1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model1.fit(X_train, Y_train,
              batch_size=batch_size, validation_data=(X_test, Y_test),
              epochs=epochs,
                shuffle=True)

Train on 1363 samples, validate on 241 samples
Epoch 1/10
1363/1363 [==============================] - 213s - loss: 1.3020 - acc: 0.6647 - val_loss: 6.8422 - val_acc: 0.5062
Epoch 2/10
1363/1363 [==============================] - 210s - loss: 0.4396 - acc: 0.7902 - val_loss: 2.8130 - val_acc: 0.4315
Epoch 3/10
1363/1363 [==============================] - 211s - loss: 0.3737 - acc: 0.8335 - val_loss: 0.9174 - val_acc: 0.5062
Epoch 4/10
1363/1363 [==============================] - 213s - loss: 0.3135 - acc: 0.8665 - val_loss: 0.6162 - val_acc: 0.6141
Epoch 5/10
1363/1363 [==============================] - 168s - loss: 0.2659 - acc: 0.8804 - val_loss: 0.9683 - val_acc: 0.5145
Epoch 6/10
1363/1363 [==============================] - 164s - loss: 0.2434 - acc: 0.9024 - val_loss: 2.0829 - val_acc: 0.5602
Epoch 7/10
1363/1363 [==============================] - 169s - loss: 0.1971 - acc: 0.9222 - val_loss: 0.4889 - val_acc: 0.7552
Epoch 8/10
1363/1363 [==============================] - 165s - l

In [8]:
model1.fit(X_train, Y_train,
              batch_size=batch_size, validation_data=(X_test, Y_test),
              epochs=5,
                shuffle=True)

Train on 1363 samples, validate on 241 samples
Epoch 1/5
1363/1363 [==============================] - 184s - loss: 0.1087 - acc: 0.9567 - val_loss: 1.5829 - val_acc: 0.6556
Epoch 2/5
1363/1363 [==============================] - 167s - loss: 0.1076 - acc: 0.9611 - val_loss: 0.3916 - val_acc: 0.8299
Epoch 3/5
1363/1363 [==============================] - 166s - loss: 0.1097 - acc: 0.9626 - val_loss: 1.3985 - val_acc: 0.6390
Epoch 4/5
1363/1363 [==============================] - 167s - loss: 0.0995 - acc: 0.9633 - val_loss: 1.8702 - val_acc: 0.7054
Epoch 5/5
1363/1363 [==============================] - 168s - loss: 0.0962 - acc: 0.9677 - val_loss: 0.4986 - val_acc: 0.7635


## convergence is fucking shit